<a href="https://colab.research.google.com/github/S-odland/ME224-projects/blob/master/automated_snake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### My Strategy

So, my basic strategy for this game was to have two arrays, x and y, which represented the positions of segments of the snake's body on the board. I found this a lot easier to do that to use the actually universe to keep tracking of taken up spaces on the board.

So I first approached this by making the board with a snake on it (hard coded start size, board size and snake placement specified in `update()`). I then moved onto just getting the snake to move and have periodic boundary conditions. After this a made a function `make_heading()` which generated a random direction and every 15 empty for loop cycles `(i in range(15))`, the snake would try to change direction. This the led me to implement the `tryput()` function which would check if there was an overlap in the heading.

After all this I implemented food and had the snake track the food. Right now the score can really only go up to 50 because the default direction of the snake is NORTH, so after almost every bite of food the snake will straighten out in the vertical direction causing the game to quit out.

A way to have prevented this would be to have the heading change if the heading specified overlaps. I tried to implement this but got a bunch of errors and gave up. But, overall, I'm very happy with how this played out.

In [ ]:
import time
import random
import sys
import numpy as np 
import matplotlib.pyplot as plt 
from IPython.display import clear_output 

# this initializes the game board with a snake on it

def make_board(n,h,w, length):

  plt.rcParams['figure.figsize'] = [10, 10]
  universe = np.zeros( (n, n), dtype=int) 

  height = universe.shape[0]
  width = universe.shape[1]

  x = list()
  y = list()

  j = 0

  for i in range(length):

    universe[h-j, w-i] = 1
    x.append(h-j)
    y.append(w-i)
  
  return x,y,height,width

# the function was for before I added food, it just generated a random heading
# 1,2,3,4 (N,E,S,W)

def make_heading():

    head = int(4 * random.random()) + 1

    return head

# randomly generates a piece of food on the game board
  
def make_food(pos,height,width):

  x = pos[0]
  y = pos[1]

  food = []
  n = 0
  i = 0

  while(i == 0):

    f_x = int(height * random.random())
    f_y = int(width * random.random())

    for i in range(len(x)):
      if (f_x == x[i] and f_y == y[i]):
        n += 1

    if (n == 0):
      i = 1
  
  return f_x, f_y

# updates the snake to 'move' in the direction of a specified heading

def go_to_heading(pos,head,food,height,width):

  x = pos[0]
  y = pos[1]

  if (head == 1):
    y_copy = y[0] + 1
    x_copy = x[0] 

    y.insert(0, y_copy)
    x.insert(0, x_copy)
    y.pop()
    x.pop()

  elif (head == 2):
    y_copy = y[0] 
    x_copy = x[0] + 1

    y.insert(0, y_copy)
    x.insert(0, x_copy)
    y.pop()
    x.pop()


  elif (head == 3):
    y_copy = y[0] - 1
    x_copy = x[0] 

    y.insert(0, y_copy)
    x.insert(0, x_copy)
    y.pop()
    x.pop()
    
  else:
    y_copy = y[0] 
    x_copy = x[0] - 1

    y.insert(0, y_copy)
    x.insert(0, x_copy)
    y.pop()
    x.pop()


  for i in range(len(x)):
    x[i] = x[i] % height
    y[i] = y[i] % width


  plt.plot( x, y, 'rs',x[0],y[0],'bs',food[0],food[1],'go',markersize = 10)
  plt.axis( [-1, width+1, -1, height+1] ) 
  plt.axis('off')
  if(head == 1):
    plt.text(width,height,'N')
  elif(head == 2):
    plt.text(width,height,'E')
  elif(head == 3):
    plt.text(width,height,'S')
  else:
    plt.text(width,height,'W')

  score = 'SCORE: ' + str(len(x)-1)
  plt.text(1,height,score)
  clear_output(True)
  plt.show()

  return x, y

# creates a heading in the direction of food

def food_heading(food,pos):

  x_dif = pos[0][0] - food[0]
  y_dif = pos[1][0] - food[1]

  new_food = 0
  head = 1


  if (abs(x_dif + food[0]) > abs(x_dif - food[0])):
    head = 4
    return head, new_food

  elif (abs(x_dif - food[0]) > abs(x_dif + food[0])):
    head = 2
    return head, new_food

  elif (abs(y_dif + food[1]) > abs(y_dif - food[1])):
    head = 3
    return head, new_food

  elif (abs(y_dif - food[1]) > abs(y_dif + food[1])):
    head = 1
    return head, new_food

  elif (x_dif == 0 and y_dif == 0):
    new_food = 1
    return head,new_food

  else:
    head = 1
    new_food = 0
    return head,new_food

# tests out specified headings and passes them on to go_to_heading() if there
# wont be an overlap

def tryput(pos,head):

  x = pos[0]
  y = pos[1]

  n = 0

  if (head == 1):
    for i in range(len(x)):
      if (y[0] + 2 == y[i] ):
        n += 1

    if(n == 0):
      return 1, head
    else:
      return 0, head
    n = 0

  elif (head == 2):
    for i in range(len(x)):
      if (x[0] + 2 == x[i] ):
        n += 1
    if(n == 0):
      return 1, head
    else:
      return 0, head      
    n = 0

  elif (head == 3):
    for i in range(len(x)):
      if (y[0] - 2 == y[i]):
        n += 1
    if(n == 0):
      return 1, head
    else:
      return 0, head
    
    n = 0

  else:
    for i in range(len(x)):
      if (x[0] - 2 == x[i] ):
        n += 1
    if(n == 0):
      return 1, head
    else:
      return 0, head
    n = 0

#culmination of the functions (makes the game)

def update():

  n = 50
  start = 25
  S = 1

  x = make_board(n,start,start,S)[0]
  y = make_board(n,start,start,S)[1]
  height = make_board(n,start,start,S)[2]
  width = make_board(n,start,start,S)[3]

  pos = [x, y]
  [f_x,f_y] = make_food(pos,height,width)

  head_prev = food_heading([f_x,f_y],pos)[0]
  pos = go_to_heading(pos,head_prev,[f_x,f_y],height,width)

  while(1):
    
    new_food = food_heading([f_x,f_y],pos)[1]

    if (new_food == 1):
      pos[0].append(pos[0][len(pos[0])-1]+1)
      pos[1].append(pos[1][len(pos[1])-1]+1)

      [f_x,f_y] = make_food(pos,height,width)

    else:

      for i in range(len(x)):
        for j in range(1,len(x)):
          if (j != i):
            if (x[i] == x[j] and y[i] == y[j]):
              sys.exit("You Lose....")

      tp = tryput(pos,food_heading([f_x,f_y],pos)[0])

      if(tp[0] == 1):
      
        head_prev = tp[1]
        pos = go_to_heading(pos,head_prev,[f_x,f_y],height,width)

      else:
        pos = go_to_heading(pos,head_prev,[f_x,f_y],height,width)
      

update()


